In [1]:
import sys
import os

path = os.path.abspath('..')
if path not in sys.path:
    sys.path.insert(0, os.path.abspath('..'))
    
sys.path

['/home/justin/Github/aqbuildtools',
 '/home/justin/Github/aqbuildtools/examples',
 '/home/justin/anaconda3/envs/aqbt/lib/python38.zip',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/lib-dynload',
 '',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages/IPython/extensions',
 '/home/justin/.ipython']

In [2]:
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['default']['aquarium']


In [ ]:
get_all_primers = {
    '__model__': 'Sample',
    '__description__': 'get all primers',
    '__query__': {
        'sample_type': {
            '__query__': {
                'name': 'Primer'
            }
        },
        '__return__': {'field_values': []}
    }
}

primers = aq.query(get_all_primers)

In [99]:
from tqdm.auto import tqdm

def pagination(query, page_size, model="Sample", pbar=True, pbar_desc=None, limit=None):
    
    if limit:
        last_id = aq.model_interface(model).first(1, query=query, opts={'offset': limit})[0].id
    else:
        last_id = aq.model_interface(model).last(1, query=query)[0].id

    opts = {}
    if limit:
        opts['limit'] = limit
    iterator = aq.model_interface(model).pagination(query=query, page_size=page_size, opts=opts)
    if pbar:
        pbar_iter = tqdm(iterator, total=last_id)
    
        if pbar_desc:
            pbar_iter.set_description(pbar_desc)
        last_max_id = -1
        for primers in pbar_iter:
 
            ids = [m.id for m in primers]
            max_id = max(ids)
            pbar_iter.update(max_id - last_max_id)
            last_max_id = max_id
            yield primers
    else:
        yield from iterator
        
def get_primers(sess, page_size=500, pbar=True, limit=None):
    all_primers = []
    for primers in pagination({'sample_type_id': sess.SampleType.find_by_name('Primer').id}, page_size=page_size, limit=limit, pbar=True, pbar_desc='collecting primer seqs'):
        sess.browser.get(primers, {'sample_type': 'field_types', 'field_values': 'field_type'})
        all_primers += primers
    return all_primers

with aq.with_cache(timeout=50) as sess:
    primers = get_primers(sess)


  0%|          | 0/36221 [00:00<?, ?it/s]

In [114]:
from json import JSONDecodeError
import re

def clean_seq(seq):
    return re.sub('\s', '', seq)

for p in primers:
    try:
        p.properties
        overhang = clean_seq(p.properties['Overhang Sequence'] or '')
        anneal = clean_seq(p.properties['Anneal Sequence'] or '')
        
        sequence = overhang + anneal
        p.overhang = overhang
        p.anneal = anneal
        p.sequence = sequence
    except JSONDecodeError as e:
        pass

In [117]:
primers[-10].overhang

''

In [118]:
import primer3plus

In [120]:
primer3plus.utils.anneal_iter?

Signature:
primer3plus.utils.anneal_iter(
    seq: str,
    primer_list: List[Union[str, Tuple[str, str]]],
    n_bases=10,
    ignore_case=True,
) -> Tuple[Iterator[Dict[str, Union[str, int]]], Iterator[Dict[str, Union[str, int]]]]
Docstring:
Anneal a list of primers to the sequence. Returns two iterables with
elements of the forms:

.. code-block:: JSON

    {
        "name": "<str> the name of the primer annealed",
        "anneal": "<str> the 5'->3' sequence of the annealing portion of the
                    primer"
        "overhang": "<str> the 5'->3' sequence of the remaining portion of the
                    primer"
        "start": "<int> the inclusive starting position of the primer"
        "length": "<int> the length of the annealing portion of the primer",
        "top_strand_slice": "tuple[int, int] the location on the top strand of the
                             annealing portion of the primer."
    }

Note the position conventions
between fwd and reverse are differe